In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    multistage_polyphase_resampler
)

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['tone'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal

In [ ]:
# test data
N = 1024
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data

# upsample and rescale
tone_8x_data = multistage_polyphase_resampler(tone_bb_data, 8.0)
tone = tone_8x_data * np.sqrt(N) # set tone mean_power to 1.0 W
tone_mean_power = np.mean(np.abs(tone)**2)
print(tone_mean_power)

In [ ]:
# examine transform response across tone input powers
pin_dBW = np.arange(-30, 10+1, 0.1) # [-30, +10] dBW input power range
pin = 10 ** (pin_dBW / 10) # input power, linear units

gain = 4.0       # linear
psat = 1.0       # dBW
phi_max = -0.1   # rad
phi_slope = 0.1  # W/rad

pow_out = np.zeros((len(pin),))
ph_out = np.zeros((len(pin),))

for i,p in enumerate(pin):
    tone_in = tone * np.sqrt(pin[i] / 1.0)              # set tone power (original 1.0 W)
    psat_backoff =  psat / np.mean(np.abs(tone_in)**2)  # set constant operating point
    response_data = F.nonlinear_amplifier(
        data = tone_in,
        gain = gain,
        psat_backoff = psat_backoff,
        phi_max = phi_max,
        phi_slope = phi_slope,
        auto_scale = False
    )
    pow_out[i] = np.mean(np.abs(response_data)**2)      # mean power output estimate
    ph_out[i] = np.mean(np.angle(response_data) - np.angle(tone_in)) # relative phase shift estimate

pin_log10 = 10*np.log10(np.where(pin == 0, float('inf'), pin))
pout_log10 = 10*np.log10(np.where(pow_out == 0, float('inf'), pow_out))

# plot pin/pout
fig = plt.figure(figsize=(8, 8))
plt.plot(pin_log10, pout_log10)
plt.title('Nonlinear Amplifier Model (AM/AM)',fontsize='large');
plt.xlabel('Pin (dBW)',fontsize='large');
plt.ylabel('Pout (dBW)',fontsize='large');
plt.show()  

# plot pin/ph_out
fig = plt.figure(figsize=(8, 8))
plt.plot(pin_log10, ph_out)
plt.title('Nonlinear Amplifier Model (AM/PM)',fontsize='large');
plt.xlabel('Pin (dBW)',fontsize='large');
plt.ylabel('$\\phi$ (rad)',fontsize='large');
plt.show()


In [ ]:
pin = np.linspace(0,10,num=10000)

mean_power_est = 1.0
psat_backoff = 1.0
gain = [0.5, 1.0, 2.0, 4.0, 8.0]

# plot pin/pout
fig = plt.figure(figsize=(8, 8))
for i,g in enumerate(gain):
    psat = mean_power_est * psat_backoff
    scale_factor = psat / g
    pout = psat * np.tanh(pin / scale_factor)
    
    pin_log10 = 10*np.log10(np.where(pin == 0, float('inf'), pin))
    pout_log10 = 10*np.log10(np.where(pout == 0, float('inf'), pout))
    plt.plot(pin_log10, pout_log10)
    
plt.title('Nonlinear Amplifier Model AM/AM across Gains ($P_{sat}$: 0 dBW)',fontsize='large');
plt.xlabel('Pin (dBW)',fontsize='large');
plt.ylabel('Pout (dBW)',fontsize='large');
plt.legend(np.round(10*np.log10(gain),decimals=1), loc='lower right', fontsize='large');
plt.show()

# plot pin/phase_out
phislope = [0.05, 0.1, 0.15, 0.2, -0.05, -0.1, -0.15, -0.2]
phi_origin = 0.5 # fix new y-axis origin at 0.5 (-3 dB)
phimax = 0.1

fig = plt.figure(figsize=(8, 8))
for i,p in enumerate(phislope):
    pin_origin = np.arctanh(phi_origin) # fix new x-axis from template tanh    
    phi = (phimax/2) * (np.tanh( (pin - pin_origin) / p) + np.sign(p))
 
    pin_log10 = 10*np.log10(np.where(pin == 0, float('inf'), pin))
    plt.plot(pin_log10, phi)

plt.title('Nonlinear Amplifier Model AM/PM across Slopes ($\\phi_{0}$, $\\phi_{max}$: (0.5, 0.1))',fontsize='large');
plt.xlabel('Pin (dBW)',fontsize='large');
plt.ylabel('$\\phi$ (rad)',fontsize='large');
plt.legend(np.round(phislope,decimals=2), loc='upper left', fontsize='large');
plt.show()